# Objaverse Downloading Script

Objaverse-XL is a Universe of 10M+ 3D Objects.
It is hosted on 🤗[Hugging Face](https://huggingface.co/datasets/allenai/objaverse-xl) and includes a [Python API on GitHub](https://github.com/allenai/objaverse-xl).

It's a bit difficult to get a dataset which is compatible for manipulation. There are a few subsets of Objaverse with additional annotations, such a Spock.

See Spock [Readme](https://github.com/allenai/spoc-robot-training) and [notebook example](https://github.com/allenai/spoc-robot-training/blob/main/how_to_use_data.ipynb)


In [ ]:
# !export OBJAVERSE_PATH="/data/lmbraid19/argusm/datasets/spok/"
# !python -m objathor.dataset.download_annotations --version 2023_07_28 --path $OBJAVERSE_PATH
# !python -m objathor.dataset.download_assets --version 2023_07_28 --path $OBJAVERSE_PATH
!pip install objaverse --upgrade --quiet
!pip install compress_json

In [ ]:
import objaverse
objaverse.__version__

# Spock Annotations
The rest of the cells are from the old tutorial

In [2]:
import pprint 
import compress_json
import objaverse
from IPython.display import Image, display
import json

pp = pprint.PrettyPrinter(indent=1)
#OBJAVERSE_ANNOTATIONS_PATH = "/data/lmbraid19/argusm/datasets/spok/2023_07_28/annotations.json.gz"
#annotations_spock = compress_json.load(OBJAVERSE_ANNOTATIONS_PATH)

OBJAVERSE_ANNOTATIONS_PATH = "/data/lmbraid19/argusm/datasets/spok/2025_01_30/annotations.json"
with open(OBJAVERSE_ANNOTATIONS_PATH) as f_obj:
    annotations_spock = json.load(f_obj)

In [ ]:
from itertools import islice
print("annotated examples", len(annotations_spock))
annotations_spock_full = annotations_spock
annotations_spock = dict(islice(annotations_spock.items(), 5000))

# Filtering
Objaverse has a ton of assets, so let's filter them a bit for sanity:
1. using existing annotation: `CanPickup`, `

In [ ]:
import numpy as np
can_pickups = []
for i, (k,v) in enumerate(iter(annotations_spock.items())):
    try:
        can_pickup = v["thor_metadata"]["assetMetadata"]["primaryProperty"] == "CanPickup"
    except:
        can_pickup = False
    can_pickups.append(can_pickup)
    #print(v['description'], v["receptacle"], )
print(np.mean(can_pickups))

In [ ]:
#lemma_set = set([v['most_specific_lemma'] for k,v in annotations_spock.items()])
#cat_set = set([v['category'] for k,v in annotations_spock.items()])

In [ ]:
print_metadata = True
if print_metadata:
    # print the metadata
    #uid = '9d17119cc0f041e39b2a11211a677366'
    uid = '584ce7acb3384c36bf252fde72063a56'
    pp.pprint(annotations_spock_full[uid])
    ann = objaverse.load_annotations([uid])
    pp.pprint(ann[uid])

In [ ]:
def find_index_with_largest_size_above_min(thubnail_image_data, min_size=448):
    valid_entries = [
        (index, entry['size']) 
        for index, entry in enumerate(thubnail_image_data) 
        if max(entry['width'], entry['height']) >= min_size
    ]
    if not valid_entries:
        return 0 # No valid entries found
    return min(valid_entries, key=lambda x: x[1])[0]

#key = '584ce7acb3384c36bf252fde72063a56'
#ann = objaverse.load_annotations([key])
#tmp = ann[key]['thumbnails']['images']
#print(find_index_with_largest_size_above_min(tmp))
#tmp
from pathlib import Path
import requests
from tqdm import tqdm_notebook as tqdm
OBJAVERSE_THUMBNAIL_PATH = Path('/data/lmbraid19/argusm/datasets/spok/thumbnails')

for i, (key, value) in tqdm(enumerate(annotations_spock.items()),total=len(annotations_spock)):
    obj_anno = annotations_spock[key]
    if "thor_metadata" not in obj_anno:
        continue
        
    bbox = obj_anno['thor_metadata']['assetMetadata']['boundingBox']
    extents = {
        'x': bbox['max']['x'] - bbox['min']['x'],
        'y': bbox['max']['y'] - bbox['min']['y'],
        'z': bbox['max']['z'] - bbox['min']['z']
    }
    extents = [extents[d]*100 for d in list('xyz')]
    if min(extents) < 2 or max(extents) > 50:
        continue
        
    ann = objaverse.load_annotations([key])
    thumbnail_image_data = ann[key]['thumbnails']['images']
    i = find_index_with_largest_size_above_min(thumbnail_image_data)
    url = thumbnail_image_data[i]['url']
    fn = OBJAVERSE_THUMBNAIL_PATH / f"{key}{Path(url).suffix}"
    if fn.is_file():
        continue

    try:
        response = requests.get(url, stream=True)  # Use streaming for large files
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.HTTPError:
        continue
    
    with open(fn, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):  # Write in chunks
            file.write(chunk)
    


## View Example Images with Annotations

In [ ]:
def print_spock_metadata(uid, metadata, extents):
    print("uid", uid)
    print("category", metadata["category"])
    print("descrip", metadata["description"])
    #print("descr-auto", metadata["description_auto"])
    print("scale", metadata["scale"], "extents"," ".join([f"{d:.0f}" for d in extents]) + " cm")

def get_extents(anno_spock):
    bbox = anno_spock['thor_metadata']['assetMetadata']['boundingBox']
    extents = {
        'x': bbox['max']['x'] - bbox['min']['x'],
        'y': bbox['max']['y'] - bbox['min']['y'],
        'z': bbox['max']['z'] - bbox['min']['z']
    }
    extents = [extents[d]*100 for d in list('xyz')]
    return extents

count = 0
for i, (key, value) in enumerate(annotations_spock.items()):
    anno_s = annotations_spock[key]
    if "thor_metadata" not in anno_s:
        continue
    
    try:
        extents = get_extents(anno_s)
    except KeyError:
        continue
    
    if min(extents) < 2 or max(extents) > 50:
        continue
        
    ann = objaverse.load_annotations([key])
    url = ann[key]['thumbnails']['images'][0]['url']
    print_spock_metadata(key, anno_s, extents)
    display(Image(url=url, width=512))
    print()

    count += 1
    #results = objaverse.load_objects([key])
    if i > 100:
        break

print("pass rate", count, "/", i)

## Select objects and download

This is done manually for now, but it should happen automatically.

In [ ]:
good_uids = ['412ed49af0644f30bae822d29afbb066',
'93128128f8f848d8bd261f6c1f763a53',
'b5c9d06f19be4c92a1708515f6655573',
'584ce7acb3384c36bf252fde72063a56',
'2796ec7a4a324d9e988d95d88bb6f1e2',
'005a246f8c304e77b27cf11cd53ff4ed',
'013176cfbc8145a0b10c3191ac265e8b',
'00bfa4e5862d4d4b89f9bcf06d2a19e4',
'088c1883e07e4946956488171e3a06bf',
'0364ab96f338493c972248102b462aa4',
'ff6c46b1e8f847ecadd5c95805f415c6']

res = objaverse.load_objects(good_uids)
print(res)

In [ ]:
#good_uids = ['b5c9d06f19be4c92a1708515f6655573','412ed49af0644f30bae822d29afbb066']

count = 0
for i, key in enumerate(good_uids):
    anno_s = annotations_spock[key]
    if "thor_metadata" not in anno_s:
        continue
    
    extents = get_extents(anno_s)
    if min(extents) < 2 or max(extents) > 50:
        continue
        
    ann = objaverse.load_annotations([key])
    url = ann[key]['thumbnails']['images'][0]['url']
    print_spock_metadata(key, anno_s, extents)
    display(Image(url=url, width=512))
    print()

    count += 1
    #results = objaverse.load_objects([key])
    if i > 100:
        break

## Render in Simulation

In [ ]:
%load_ext autoreload
%autoreload 2
import gymnasium as gym
import mani_skill.examples.clevr_env  # do import to register env, not used otherwise
from mani_skill.envs.sapien_env import BaseEnv
from mani_skill.examples.gen_dataset import Args, reset_random
import matplotlib.pyplot as plt 
    
args = Args()
args.env_id = "ClevrMove-v1"
#args.shader = "rt-fast" or "rt"
#args.seed = [42,]
reset_random(args)

vis = False
env: BaseEnv = gym.make(
    args.env_id,
    obs_mode=args.obs_mode,
    reward_mode=args.reward_mode,
    control_mode=args.control_mode,
    render_mode=args.render_mode,
    sensor_configs=dict(shader_pack=args.shader),
    human_render_camera_configs=dict(shader_pack=args.shader),
    viewer_camera_configs=dict(shader_pack=args.shader),
    num_envs=args.num_envs,
    sim_backend=args.sim_backend,
    parallel_in_single_scene=False,
    robot_uids="panda_wristcam",
    object_dataset="objaverse",
    scene_dataset="Table",
    # **args.env_kwargs
)

for i in range(10**6):
    obs, _ = env.reset(seed=args.seed[0], options=dict(reconfigure=True))
    if args.seed is not None:
        env.action_space.seed(args.seed[0])
    if vis and args.render_mode is not None:
        viewer = env.render()
        if isinstance(viewer, sapien.utils.Viewer):
            viewer.paused = args.pause
        env.render()
    else:
        env.render()
    
    # get before image
    images = env.base_env.scene.get_human_render_camera_images('render_camera')
    print(i)
    plt.imshow(images['render_camera'][0].detach().cpu().numpy())
    plt.show()
    if i > 0:
        break
    reset_random(args)

# [Old-Stuff] Load all objaverse UIDs

In [ ]:
uids = objaverse.load_uids()
len(uids), type(uids)

In [ ]:
# This takes a couple of minutes
annotations = objaverse.load_annotations(uids)

In [ ]:
import pprint 
for i, (key, annotation) in enumerate(annotations.items()):
    pp.pprint(annotation)
    #print(annotation["description"])
    if i > 2:
        break

# [Old-Stuff] LIVS Annotation Stuff

LVIS was this subset of objaverse which hand proper annotations. I was looking at it before I found the Spock annotations.

In [ ]:
import objaverse
counts = [(k,len(v)) for k,v in annotations2.items()]

In [ ]:
# select N_categories objects
N_categories = 100
print(len(counts))
sum([v for k, v in counts])
collection = []
i = 0
for i, (k, v) in enumerate(annotations2.items()):
  if i >= N_categories:
    break
  collection.append(v[0])
print(collection)

In [ ]:
results = objaverse.load_objects(collection)

### Object loading code, LVIS names

In [ ]:
# glb_downloaded = [x.stem for x in glb_files]
# glb_downloaded_set = set(glb_downloaded)            
# anno_path = objaverse_folder / "lvis-annotations.json.gz"
# import gzip, json
# with gzip.open(anno_path, "rt", encoding="utf-8") as f:
#     lvis_annotations = json.load(f)
# mapping = {}
# for category, items in lvis_annotations.items():
#     for index, item in enumerate(items, start=1):  # Use 1-based indexing
#         if item not in glb_downloaded_set:
#             continue
#         mapping[f"{category}-{index}"] = glb_files[glb_downloaded.index(item)]
# self.objaverse_model_ids = list(mapping.keys())
# self.objaverse_files = mapping